In [35]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")


display(x_train.head())
display(y_train.head())

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


In [36]:
x_train.nunique()

,0
CustomerId,6499
Surname,2289
CreditScore,459
Geography,3
Gender,4
Age,69
Tenure,11
Balance,4162
NumOfProducts,4
HasCrCard,2


In [37]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 609.4+ KB


In [38]:
cols = ['Gender', 'Geography']

from sklearn.preprocessing import LabelEncoder

for col in cols:
    le = LabelEncoder();
    x_train[col]=le.fit_transform(x_train[col])
    x_test[col]=le.fit_transform(x_test[col])

x_train = x_train.drop('Surname', axis=1)
x_test = x_test.drop('Surname', axis=1)

In [39]:
x_train = x_train.drop('CustomerId', axis=1)
y_train = y_train.drop('CustomerId', axis=1)
test_id = x_test.pop('CustomerId')

In [40]:
x_train.shape, x_test.shape, y_train.shape

((6499, 10), (3501, 10), (6499, 1))

In [41]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2022)

In [42]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_tr, y_tr)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [43]:
pred = model.predict(X_val)
pred_proba = model.predict_proba(X_val)

from sklearn.metrics import accuracy_score , f1_score, recall_score, roc_auc_score ,precision_score
print(accuracy_score(y_val, pred))
print(f1_score(y_val, pred))
print(recall_score(y_val, pred))
print(roc_auc_score(y_val, pred_proba[:, 1]))
print(precision_score(y_val, pred))

0.8546153846153847
0.548926014319809
0.42124542124542125
0.8459327105870436
0.7876712328767124


In [44]:
pred = model.predict(x_test)
pred_proba = model.predict_proba(x_test)

y_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv")
y=y_test['Exited']

from sklearn.metrics import accuracy_score , f1_score, recall_score, roc_auc_score ,precision_score
print(accuracy_score(y, pred))
print(f1_score(y, pred))
print(recall_score(y, pred))
print(roc_auc_score(y, pred_proba[:, 1]))
print(precision_score(y, pred))

0.8603256212510711
0.5550500454959054
0.4277699859747546
0.8480003964093761
0.7901554404145078


In [45]:
submit = pd.DataFrame({
    'CustomerId':test_id,
    'Exited':pred
})

submit.to_csv("000000.csv", index=False)